In [ ]:
import getopt
import sys
import pandas as pd
from pfs.lam.detAnalysis import *
import time

In [ ]:
from pfs.lam.opdb import *


In [ ]:
from pfs.lam.fileHandling import *
from pfs.lam.detFocusAnalysis import *
import os
import lsst.daf.persistence as dafPersist
import re
from astropy.modeling import models, fitting
from astropy.stats import gaussian_sigma_to_fwhm

In [ ]:
def VisitImageQualityToCsv(visit, \
                           peak_list, roi_size, com=True, doBck=True, EE=[3,5], doFit=True, doLSF=False,\
                           cam=None, repo="cluster",rerun="sm1-dither", cluster=False, doPlot=False,\
                           doSep=False, mask_size=50, threshold= 50, subpix = 5 , maxPeakDist=80,\
                           maxPeakFlux=40000, minPeakFlux=2000,\
                          csv_path = "", drpImage=None, experimentId = None, doPrint=False):
    if drpImage is None:
        experimentId = Logbook.visitExperimentId(visit=visit)
        
        doRaise = True
        try:
            filepath, date = getvisitfilepath(visit, rerun, None, cam, repo, cluster=cluster)
            visitfilepath = filepath
        except IOError:
            if doRaise:
                raise
    else :
        filepath = drpImage[0]
        visitfilepath = drpImage[1]
        visit = drpImage[2]
        try:
            #experimentId = Logbook.visitExperimentId(visit=visit)
            experimentId = get_Visit_Set_Id(visit)
        except:
            experimentId = np.nan
            print("Unable to get experimentId from logbook")

    data = getImageQuality(filepath, peak_list, roi_size=roi_size, EE=EE, com=com, doPlot=doPlot, doBck=doBck, doFit=doFit, doLSF=doLSF)
    if doSep:
        dsep = getImageEncerclEnergy(filepath, peak_list, roi_size=roi_size, EE=EE,\
        mask_size=mask_size, threshold= threshold, subpix = subpix ,\
        maxPeakDist=maxPeakDist, maxPeakFlux=maxPeakFlux, minPeakFlux=minPeakFlux,\
        doPlot=doPlot, doBck=doBck)
        dsep = dsep.add_prefix("sep_")
        dsep = dsep.rename(columns={'sep_peak': 'peak','sep_fiber': 'fiber'})
        data = data.merge(dsep, on=["peak","fiber"]) 
    __, fname = os.path.split(visitfilepath)

    data["filename"] = fname
    data["visit"] = visit

#    data["peaklist"] = peak_list        

#    data["motor1"] = np.float(getFitsKey(visitfilepath, 'W_XCU_MOTOR1_MICRONS'))
#    data["motor2"] = np.float(getFitsKey(visitfilepath, 'W_XCU_MOTOR2_MICRONS'))
#    data["motor3"] = np.float(getFitsKey(visitfilepath, 'W_XCU_MOTOR3_MICRONS'))
    data["xm1pos"] = np.float(getFitsKey(visitfilepath, 'W_XM1POS'))
    data["xm2pos"] = np.float(getFitsKey(visitfilepath, 'W_XM2POS'))
    data["xm3pos"] = np.float(getFitsKey(visitfilepath, 'W_XM3POS'))
    data["motor1"] = data["xm1pos"] 
    data["motor2"] = data["xm2pos"]
    data["motor3"] = data["xm3pos"]   
    fcax = np.float(getFitsKey(visitfilepath, 'W_ENFCAX', doRaise=False))
    fcay = np.float(getFitsKey(visitfilepath, 'W_ENFCAY', doRaise=False))
    fcaz = np.float(getFitsKey(visitfilepath, 'W_ENFCAZ', doRaise=False))

    # OneChannel back compatiblity
    fcax = np.float(getFitsKey(visitfilepath, 'HIERARCH W_FCA_FOCUS', doRaise=False)) if np.isnan(fcax) else fcax
    data['fcaFocus'] = fcax
    data['fcaY'] = fcay
    data['fcaZ'] = fcaz

#    feeTemp = np.float(getFitsKey(visitfilepath, 'temps.FEE'))
#    feeTemp = np.float(getFitsKey(visitfilepath, 'HIERARCH temps.FEE', doRaise=False)) if np.isnan(feeTemp) else feeTemp

#    data['feeTemp'] = feeTemp
#    ccdTemp = np.float(getFitsKey(visitfilepath, 'temps.CCD0'))
#    ccdTemp = np.float(getFitsKey(visitfilepath, 'HIERARCH temps.CCD0', doRaise=False)) if np.isnan(ccdTemp) else ccdTemp
#    data['ccdTemp'] = ccdTemp
    ccdTemp = np.float(getFitsKey(visitfilepath, 'W_XTDET1', doRaise=False))
    data['ccdTemp'] = ccdTemp
    detBoxTemp = np.float(getFitsKey(visitfilepath, 'W_XTDBOX', doRaise=False))
    data['detBoxTemp'] = detBoxTemp

    data['cam'] = cam
    data['obsdate'] = getFitsKey(visitfilepath, 'DATE-AVG')
    data['experimentId'] = experimentId

    now = datetime.now() # current date and time\n",
    date_time = now.strftime("%Y%m%dT%Hh%M")

    csvName = f"Imquality_{cam}_Exp{experimentId}_{visit}_{date_time}.csv"
    if not os.path.exists(csv_path):
        os.makedirs(csv_path)
    data.to_csv(csv_path+csvName)
    if doPrint:
        print(csv_path+csvName)

    return

In [ ]:
def main(argv):
    visit = ''
    peak = ''
    outpath = ''
    try:
        opts, args = getopt.getopt(argv,"hv:p:c:o:r:",["visit=","peak=", "cam=", "outpath=", "rerun="])
    except getopt.GetoptError:
        print('Cluster_GetIMqual.py -v <visit> -p <peakfile> -c <cam> -r <rerun>')
        sys.exit(2)
    for opt, arg in opts:
        if opt == '-h':
            print('Cluster_GetIMqual.py -v <visit> -p <peakfile> -c <cam>')
            sys.exit()
        elif opt in ("-v", "--visit"):
            visit = int(arg)
        elif opt in ("-p", "--peak"):
            peaklist = arg
        elif opt in ("-c", "--cam"):
            cam = arg
        elif opt in ("-o", "--outpath"):
            outpath = arg
        elif opt in ("-r", "--rerun"):
            rerun = arg
           


    outpath = "output\\" if outpath is None else outpath

    # define defaut parameters
    roi_size = 24
    com = True  # Center Of Mass
    doBck = True
    head = 0
    tail = 0
    criteria = 'EE5'
    verbose = False
    doPrint = False
    arm = cam[0]
    
    drpPath = "/drp/"
    repo = "sps"
    basePath = '/drp/fmadec/analysis/sm2/'
    extra = ''
    
    try:
#            experimentId = Logbook.visitExperimentId(visit=visit)
            experimentId = get_Visit_Set_Id(visit)

    except:
            experimentId = np.nan
            print("Unable to get experimentId from logbook")
            
    csvPath = basePath+"Exp"+str(experimentId)+"/"+rerun+"_roi"+str(roi_size)+"/doBck"+str(doBck)+"/"+extra
    print(csvPath)
    
#    peakPath ="/home/fmadec/dev/ait-notebook/optical/input/"
#    peaklist = peakPath+peaklist
#    peaklist = "/drp/cluster/devel/ait-notebook/fmadec/Subaru/SM1/selectLines/Imqual_LAM_peaklist_2020July.csv"
    peaklist = "SM2_peak_engineering_20201210.csv"

#   VisitImageQualityToCsv(visit, peaklist, roi_size, cam=cam, \
#                          repo="sm1",rerun="ginga",  cluster=True, \
#                          com=True, doBck=True, EE=[3,5], doFit=True, doLSF=False,
#                          csv_path = outpath)

    print(f'Start visit {visit} of {cam} with {peaklist}')
    print(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")
    
    butler = dafPersist.Butler(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")
    rbutler = dafPersist.Butler(f"{drpPath}/{repo}/")
    rbutler.getKeys('raw')
    lamps = rbutler.queryMetadata('raw', ['lamps'], visit=visit, arm=arm) 
    calExp = butler.get("calexp", visit=visit, arm=cam[0])

    peaks = pd.read_csv(peaklist)
    selected_peaks = peaks[peaks.arm == arm]
    selected_peaks = selected_peaks[(selected_peaks.element.str.contains('|'.join(re.findall('[A-Z][^A-Z]*', "".join(lamps)))))]
    
    visit = int(visit)
    calExp = butler.get("calexp", visit=visit, arm=cam[0])
    calexfilePath = butler.getUri("calexp", visit=visit, arm=cam[0])
    if verbose:
        print(calexfilePath)
    
    VisitImageQualityToCsv(visit, selected_peaks, roi_size,\
                           drpImage=[ calExp.image.array, calexfilePath, visit],\
                           com=com, doBck=doBck, EE=[3,5], doFit=True, doLSF=True,\
                           cam=cam, repo=None,rerun=None, cluster=False, doPlot=False,\
                           doSep=True, mask_size=20, threshold= 50, subpix = 5 , maxPeakDist=80,\
                           maxPeakFlux=40000, minPeakFlux=2000,\
                          csv_path = csvPath, experimentId=experimentId, doPrint=verbose)

In [ ]:
experimentId = 124
cam = "b2"
rerun = "defocus" #"dcb2"

peaklist = "Imqual_LAM_peaklist_DEBUG_Neon_3peaks_B2_Dec2020.csv"


In [ ]:
experimentId = 125
cam = "b2"
rerun = "defocus" #"dcb2"

peaklist = "../SM2-defocus/Imqual_LAM_peaklist_DCB2-notfinal.csv"


In [ ]:
experimentId = 136
cam = "r2"
rerun = "defocus" #"dcb2"

peaklist = "Imqual_LAM_peaklist_DEBUG_HgAr_R2_Dec2020.csv"


In [ ]:
experimentId = 152
cam = "r2"
rerun = "defocus" #"dcb2"

peaklist = "Imqual_LAM_peaklist_DEBUG_HgAr_R2_Dec2020.csv"


In [ ]:
experimentId = 153
cam = "r2"
rerun = "defocus" #"dcb2"

peaklist = "Imqual_LAM_peaklist_DEBUG_HgAr_R2_Dec2020.csv"


In [ ]:
experimentId = 154
cam = "r2"
rerun = "defocus" #"dcb2"

peaklist = "Imqual_LAM_peaklist_DEBUG_HgAr_R2_Dec2020.csv"


In [ ]:
visitStart, visitEnd = getVisitRange(experimentId)

In [ ]:
print(f"visit: {visitStart}-{visitEnd}")

In [ ]:
get_Visit_Set_Id(visitStart)

In [ ]:
# define defaut parameters
roi_size = 24
com = True  # Center Of Mass
doBck = True
head = 0
tail = 0
criteria = 'EE5'
verbose = True
doPrint = False
arm = cam[0]

drpPath = "/drp"
repo = "sps"
basePath = '/drp/analysis/sm2/'
extra = ''

for visit in range(visitStart, visitEnd+1):

    now = datetime.now() # current date and time\n",
    date_time = now.strftime("%Y%m%dT%Hh%M")
    csvPath = basePath+"Exp"+str(experimentId)+"/"+rerun+"_roi"+str(roi_size)+"/doBck"+str(doBck)+"/"+extra+date_time+"/"
    print(csvPath)

    print(f'Start visit {visit} of {cam} with {peaklist}')
    print(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")

    butler = dafPersist.Butler(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")
    rbutler = dafPersist.Butler(f"{drpPath}/{repo}/")
    rbutler.getKeys('raw')
    lamps = rbutler.queryMetadata('raw', ['lamps'], visit=visit, arm=arm) 
    print(f"lamps used: {lamps}")
    calExp = butler.get("calexp", visit=visit, arm=cam[0])

    peaks = pd.read_csv(peaklist)
    selected_peaks = peaks[peaks.arm == arm]
    selected_peaks = selected_peaks[(selected_peaks.element.str.contains('|'.join(re.findall('[A-Z][^A-Z]*', "".join(lamps)))))]
    #selected_peaks = peaks
    print(selected_peaks.element.unique())
    visit = int(visit)
    calExp = butler.get("calexp", visit=visit, arm=cam[0])
    calexfilePath = butler.getUri("calexp", visit=visit, arm=cam[0])
    if verbose:
        print(calexfilePath)

    VisitImageQualityToCsv(visit, selected_peaks, roi_size,\
                           drpImage=[ calExp.image.array, calexfilePath, visit],\
                           com=com, doBck=doBck, EE=[3,5], doFit=True, doLSF=True,\
                           cam=cam, repo=None,rerun=None, cluster=False, doPlot=False,\
                           doSep=True, mask_size=20, threshold= 50, subpix = 5 , maxPeakDist=80,\
                           maxPeakFlux=40000, minPeakFlux=2000,\
                          csv_path = csvPath, experimentId=experimentId, doPrint=verbose)

In [ ]:

print(date_time)